# 1  The Environment

The environment for this project is a graph of nodes connected by edges.  The agent, the prey, and the predator,can move between the nodes along the edges.  There are 50 nodes, numbered 1 to 50, connected in a large circle.Additionally, add edges at random to increase connectivity across the circle, in the following way:•Picking a random node with degree less than 3,•add an edge between it and one node within 5 steps forward or backward along the primary loop.  (So node 10might get connected to node 7 or node 15, but not node 16.)•Do this until no more edges can be added.

Graph Theory Question:With  this  setup,  you  can  add  at  most  25  additional  edges  (why?).   Are  you always able to add this many?  If not, what’s the smallest number of edges you’re always able to add?

In [2]:
import random
import copy
def create_graph():

  # create nodes
  graph = {}
  node_degree = {}
  for i in range(1,51):
    node_degree[i] = 0
    graph[i] = []


  #connect them in a circle
  for i in range(1,51):
    graph[i].append((i%50)+1)
    graph[(i%50)+1].append(i)
    node_degree[i] += 1
    node_degree[(i%50)+1] += 1

  #print(graph)

  # helper to compute all +5/-5 radius vertice from current vertex
  def getfb(x):
    possible_nodes = []
    for i in range(1,6):
      fe = x+i
      be = x-i

      if fe>50:
        fe = fe%50
      if be<1:
        be = 50+be
      possible_nodes.append(fe)
      possible_nodes.append(be)
    return possible_nodes

  # helper function to check whether more nodes can be added to graph
  def possible_to_add_more_edges(graph,node_degree):
    valid_nodes = [k for k,v in graph.items() if len(v) < 3]
    for k in valid_nodes:
      if node_degree[k]<3:
        possible_nodes  = getfb(k)
        #print(k,possible_nodes)
        for i in possible_nodes:
          if node_degree[i]<3 and i not in graph[k]:
            return True
    return False

  c = 0

  # add edges +5/-5 from the current vertices
  while(possible_to_add_more_edges(graph,node_degree)):
    #print('yay')
    valid_nodes = [k for k,v in graph.items() if len(v) < 3]
    t = random.randint(0,len(valid_nodes)-1)
    if(len(graph[valid_nodes[t]])<3):
      possible_nodes = getfb(valid_nodes[t])
      x = random.randint(0,len(possible_nodes)-1)
      if node_degree[possible_nodes[x]] < 3 and possible_nodes[x] not in graph[valid_nodes[t]]:
        graph[possible_nodes[x]].append(valid_nodes[t])
        graph[valid_nodes[t]].append(possible_nodes[x])
        node_degree[possible_nodes[x]] += 1
        node_degree[valid_nodes[t]] += 1
        c += 1
        

  return graph, node_degree, c

Graph Theory Question:With  this  setup,  you  can  add  at  most  25  additional  edges  (why?).   Are  youalways able to add this many?  If not, what’s the smallest number of edges you’re always able to add?

In [3]:
# '''
# t = 10000
# m = 25
# while(t):
#   graph, node_degree, c = create_graph()
#   m = min(m,c)
#   t -= 1
# print(m)
# '''


# Initialise Prey, pred and agent

In [19]:
def initilise_prey_pred_agent():
  nodes = [i for i in range(1,51)]
  current_agent_pos = random.choice(nodes)
  nodes.remove(current_agent_pos)
  current_prey_pos = random.choice(nodes)
  current_pred_pos = random.choice(nodes)
  return current_agent_pos,current_prey_pos,current_pred_pos

# Modeling Prey and Predator

The Prey: The rules for the Prey are simple - every time the Prey moves, it selects among its neighbors or its current cell, uniformly at random (i.e., if it has 3 neighbors, there is a 1/4 probability of it staying where it is.)  This continues, regardless of the actions or locations of the others, until the game concludes.

In [14]:
def move_prey(graph, current_prey_pos):
  temp = graph[current_prey_pos].copy()
  temp.append(current_prey_pos)
  t = random.randint(0,len(temp)-1)
  next_move = temp[t]
  return next_move



The Predator: Every time the Predator moves,  it looks at its available neighbors,  and calculates the shortest distance  to  the  Agent  for  each  neighbor  it  can  move  to.   It  then  moves  to  the  neighbor  with  shortest  distance remaining to the Agent.  If multiple neighbors have the same distance to the Agent, the Predator selects uniformly at random between them.

In [15]:
def BFS(graph, src, dest, pred, dist):

  queue = []
  visited = [False for i in range(len(graph)+1)]

  #initializing dist and predecessor array
  for i in range(len(graph)+1):
    dist[i] = 1000000
    pred[i] = -1

  visited[src] = True
  dist[src] = 0
  queue.append(src)

  # BFS
  while (len(queue) != 0):
    u = queue[0]
    queue.pop(0)
    for i in range(len(graph[u])):
      if (visited[graph[u][i]] == False):
        visited[graph[u][i]] = True
        dist[graph[u][i]] = dist[u] + 1
        pred[graph[u][i]] = u
        queue.append(graph[u][i])

        # stop when destination is found
        if (graph[u][i] == dest):
          return True

  return False


def calculate_shortest_distance(graph, src, dest):

  if src==dest:
    return 0, []

  pred=[0 for i in range(len(graph)+1)]
  dist=[0 for i in range(len(graph)+1)]

  if (BFS(graph, src, dest, pred, dist) == False):
    print("BFS returns false.\nGraph =",graph,"\nSrc = ",src,"\nDest = ",dest,"\nPred = ",pred,"\nDist = ",dist)
    return False

  # storing path 
  path = []
  crawl = dest
  path.append(crawl)

  while (pred[crawl] != -1):
    path.append(pred[crawl])
    crawl = pred[crawl]

  min_dist = dist[dest]
  path.reverse()

  return min_dist, path 

  
#graph, node_degree, c = create_graph()
#a, b = calculate_shortest_distance(graph, 1, 50)
#print(a,b)




In [16]:
# '''
# all_shortest_distances = {}
# for i in range(1,51):
#   for j in range(1,51):
#     all_shortest_distances[(i,j)] = calculate_shortest_distance(graph, i, j)

# #all_shortest_distances

# '''

In [17]:
def move_predator(graph, current_pred_pos, current_agent_pos):
  temp = graph[current_pred_pos] + [current_pred_pos]
  dist = []
  
  for i in temp:
    if(calculate_shortest_distance(graph ,i, current_agent_pos)):
      x, y = calculate_shortest_distance(graph ,i, current_agent_pos)
      dist.append(x)
    else:
      dist.append(100000)
  shortest_distance = min(dist)
  possible_moves = []
  for i in range(len(temp)):
    if dist[i]==shortest_distance:
      possible_moves.append(temp[i])
  
  if len(possible_moves)>1:
    return possible_moves[random.randint(0,len(possible_moves)-1)]
  
  return possible_moves[0]

 

THE EASILY DISTRACTED PREDATOR

Instead of always moving to close the distance, the predator gets easily distracted - with probability 0.6, the predator will move to close the distance, but with probability 0.4, the predator moves to one of its neighbors uniformly at random. This means that on average the predator is moving to close the distance, but it is not guaranteed in any round.

In [18]:
import random
def move_distracted_predator(graph, current_pred_pos, current_agent_pos):
  c = random.uniform(0, 1)
  temp = graph[current_pred_pos] 

  if (c<=0.6):
    dist = []
    
    for i in temp:
      if(calculate_shortest_distance(graph ,i, current_agent_pos)):
        x, y = calculate_shortest_distance(graph ,i, current_agent_pos)
        dist.append(x)
      else:
        dist.append(100000)
    shortest_distance = min(dist)
    possible_moves = []
    for i in range(len(temp)):
      if dist[i]==shortest_distance:
        possible_moves.append(temp[i])
    
    if len(possible_moves)>1:
      return possible_moves[random.randint(0,len(possible_moves)-1)]
    return possible_moves[0]
  else:
    return temp[random.randint(0,len(temp)-1)]


# Base Agent

In [10]:
# def move_agent0(graph, curr_agent_pos, prey_pos):
  # dist_c_prey, x = calculate_shortest_distance(graph ,curr_agent_pos, prey_pos)
  # dist_from_prey = []
  # for i in range(len(graph[curr_agent_pos])):
  #   t1, t2 = calculate_shortest_distance(graph ,graph[curr_agent_pos][i], prey_pos)
  #   dist_from_prey.append(t1)
  # dist_from_prey.append(dist_c_prey)
  # min_prey_dist = min(dist_from_prey)
  # temp = list(zip(dist_from_prey,graph[curr_agent_pos]))
  # temp = [x[1] for x in temp if x[0]==min_prey_dist]
  # return random.choice(temp)
p1,p2 = [0,0]
def move_agent0(graph, curr_agent_pos, pred_pos):
  dist_c_pred, x = calculate_shortest_distance(graph ,curr_agent_pos, pred_pos)
  dist_from_pred = []
  for i in range(len(graph[curr_agent_pos])):
    t1, t2 = calculate_shortest_distance(graph ,graph[curr_agent_pos][i], pred_pos)
    dist_from_pred.append(t1)
  dist_from_pred.append(dist_c_pred)
  nodes = copy.deepcopy(graph[curr_agent_pos])
  nodes.append(curr_agent_pos)
  # print("dist_from_pred:",dist_from_pred)
  max_pred_dist = max(dist_from_pred)
  # print("max_pred_dist:",max_pred_dist)
  temp = list(zip(dist_from_pred,nodes))
  temp = [x[1] for x in temp if x[0]==max_pred_dist]
  # print("temp=",temp)
  # if (len(temp)>1):
  temp = random.choice(temp)
  # print("return_temp=",temp)
  return temp


  

In [11]:
def agent0( current_agent_pos , current_prey_pos , current_pred_pos):
  hangs = 0

  while (True):
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      print("all 3 in same node, nobody wins!")
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    current_agent_pos = move_agent0(graph, current_agent_pos, current_pred_pos)
    hangs += 1
    if (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
    elif (hangs > t_hangs):
      return("agent hangs!",[current_agent_pos, current_prey_pos, current_pred_pos])
    
    current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos) #add the prey moves to list
    if (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
    
    current_pred_pos = move_predator(graph, current_pred_pos, current_agent_pos)



# Agent1

The Complete Information Setting:In this setting, the Agent always knows exactly where the Predator is and where the Prey is.

–Agent 1:

Whenever it is this Agent’s turn to move, it will examine each of its available neighbors andselect from them in the following order (breaking ties at random).

∗Neighbors that are closer to the Prey and farther from the Predator.

∗Neighbors that are closer to the Prey and not closer to the Predator.

∗Neighbors that are not farther from the Prey and farther from the Predator.

∗Neighbors that are not farther from the Prey and not closer to the Predator.

∗Neighbors that are farther from the Predator.

∗Neighbors that are not closer to the Predator.

∗Sit still and pray.

In [22]:
def move_agent1(graph, curr_agent_pos, prey_pos, pred_pos):
  dist_c_prey, x = calculate_shortest_distance(graph ,curr_agent_pos, prey_pos)
  dist_c_pred, x = calculate_shortest_distance(graph ,curr_agent_pos, pred_pos)
  dist_from_prey = []
  dist_from_pred = []
  global p1,p2 
  for i in range(len(graph[curr_agent_pos])):
    t1, t2 = calculate_shortest_distance(graph ,graph[curr_agent_pos][i], prey_pos)
    dist_from_prey.append(t1)
    t1, t2 = calculate_shortest_distance(graph ,graph[curr_agent_pos][i], pred_pos)
    dist_from_pred.append(t1)
  p1,p2 = random.uniform(30*30, 63),random.uniform(65.0, 67.5)

  #Neighbors that are closer to the Prey and farther from the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000

  for i in range(len(graph[curr_agent_pos])):
    if dist_from_prey[i]<dist_c_prey and dist_from_pred[i]>dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    # print("1:",d_n)
    # global p1,p2
    d_n.sort()
    d_n = [x for x in d_n if x[0][0]==min_prey]
    return d_n[len(d_n)-1][1]

  #Neighbors that are closer to the Prey and not closer to the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_prey[i]<dist_c_prey and dist_from_pred[i]==dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    # print("2:",d_n)
    # global p1,p2
    d_n.sort()
    d_n = [x for x in d_n if x[0][0]==min_prey]
    return d_n[0][1]

  #Neighbors that are not farther from the Prey and farther from the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_prey[i]==dist_c_prey and dist_from_pred[i]>dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    # print("3:",d_n)
    d_n.sort()
    d_n = [x for x in d_n if x[0][0]==min_prey]
    return d_n[len(d_n)-1][1]

  #Neighbors that are not farther from the Prey and not closer to the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_prey[i]==dist_c_prey and dist_from_pred[i]==dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    # print("4:",d_n)
    return d_n[random.randint(0,len(d_n)-1)][1]

  #Neighbors that are farther from the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_pred[i]>dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    # print("5:",d_n)
    return d_n[random.randint(0,len(d_n)-1)][1]
  
  #Neighbors that are not closer to the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_pred[i]==dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    # print("6:",d_n)
    return d_n[random.randint(0,len(d_n)-1)][1]

  #Sit still and pray.
  # print("7:",curr_agent_pos)
  return curr_agent_pos


In [21]:
def agent1( current_agent_pos , current_prey_pos , current_pred_pos):
  hangs = 0
  i=0

  while (True):
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    current_agent_pos = move_agent1(graph, current_agent_pos, current_prey_pos, current_pred_pos)
    hangs += 1
    if (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos])
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
    elif (hangs > t_hangs):
      return("agent hangs!",[current_agent_pos, current_prey_pos, current_pred_pos])
    
    # if(i < len(prey_pos_list)):
    #   current_prey_pos = prey_pos_list[i]
    #   i += 1
    # else:
    current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos) #add the prey moves to list
    if (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])

    current_pred_pos = move_predator(graph, current_pred_pos, current_agent_pos)




# Agent 2

In [23]:
def move_agent2(graph, curr_agent_pos, prey_pos, pred_pos):
  dist_c_prey, dist_c_prey_path = calculate_shortest_distance(graph ,curr_agent_pos, prey_pos)
  dist_c_pred, dist_c_pred_path = calculate_shortest_distance(graph ,curr_agent_pos, pred_pos)
  dist_from_prey = []
  dist_from_pred = []
  dist_from_prey_path = []
  dist_from_pred_path = []
  temp_prey_pred = []
  for i in range(len(graph[curr_agent_pos])):
    t1, t2 = calculate_shortest_distance(graph ,graph[curr_agent_pos][i], prey_pos)
    dist_from_prey.append(t1)
    dist_from_prey_path.append(t2)
    t1, t2 = calculate_shortest_distance(graph ,graph[curr_agent_pos][i], pred_pos)
    dist_from_pred.append(t1)
    dist_from_pred_path.append(t2)
    temp_prey_pred.append((dist_from_prey[-1],dist_from_pred[-1]))


  temp_zip = list(zip(temp_prey_pred,graph[curr_agent_pos]))

  min_prey = min(dist_from_prey)
  max_pred = max(dist_from_pred)


  #Neighbors that are closer to the Prey and farther from the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000

  for i in range(len(graph[curr_agent_pos])):
    if dist_from_prey[i]<dist_c_prey and dist_from_pred[i]>dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])

  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    d_n.sort()
    d_n = [x for x in d_n if x[0][0]==min_prey]
    return d_n[len(d_n)-1][1]

  #if the pred is less than 5 steps away from agent and the distance to prey is greater than distance to pred and is the dist to pred is greater than the current nodes dist to pred then move to that node.
  if(dist_c_prey > dist_c_pred and dist_c_pred < 3  and max_pred > dist_c_pred):
    node = [x[1] for x in temp_zip if x[0][1]==max_pred]
    node = node[random.randint(0,len(node)-1)]
    return(node)

  #Neighbors that are closer to the Prey and not closer to the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_prey[i]<dist_c_prey and dist_from_pred[i]==dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    d_n.sort()
    d_n = [x for x in d_n if x[0][0]==min_prey]
    return d_n[0][1]

  #Neighbors that are not farther from the Prey and farther from the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_prey[i]==dist_c_prey and dist_from_pred[i]>dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    d_n.sort()
    d_n = [x for x in d_n if x[0][0]==min_prey]
    return d_n[len(d_n)-1][1]

  #Neighbors that are not farther from the Prey and not closer to the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_prey[i]==dist_c_prey and dist_from_pred[i]==dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    return d_n[random.randint(0,len(d_n)-1)][1]

  #Neighbors that are farther from the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_pred[i]>dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    return d_n[random.randint(0,len(d_n)-1)][1]
  
  #Neighbors that are not closer to the Predator.
  temp_node = []
  temp_prey_pred = []
  min_prey = 10000
  for i in range(len(graph[curr_agent_pos])):
    if dist_from_pred[i]==dist_c_pred:
      temp_node.append(graph[curr_agent_pos][i])
      temp_prey_pred.append((dist_from_prey[i],dist_from_pred[i]))
      min_prey = min(min_prey,dist_from_prey[i])
  if len(temp_node)>0:
    d_n = list(zip(temp_prey_pred,temp_node))
    return d_n[random.randint(0,len(d_n)-1)][1]

  #Sit still and pray.
  return curr_agent_pos
    


In [24]:
import numpy
def agent2( current_agent_pos , current_prey_pos , current_pred_pos):
  hangs = 0
  i=0
  while (True):
    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos])

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    current_agent_pos = move_agent2(graph, current_agent_pos, current_prey_pos, current_pred_pos)
    hangs += 1
    if (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])
    elif (hangs > t_hangs):
      return("agent hangs!",[current_agent_pos, current_prey_pos, current_pred_pos])
    
    if(i < len(prey_pos_list)):
      current_prey_pos = prey_pos_list[i]
      i += 1
    else:
      current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos) #add the prey moves to list
    if (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos])

    current_pred_pos = move_predator(graph, current_pred_pos, current_agent_pos)


# Agent 3

In [25]:
def update_belief_survey(beliefs, t, state, graph, node_degree, pos_where_prey_not_there):
  new_beliefs = [0]*len(beliefs)
  if state == True: # found the prey
    new_beliefs[t] = 1
    for i in range(len(beliefs)):
      if i!=t:
        new_beliefs[i]=0

  if state == False: # did not find the prey
    # P(prey in nodes| survey failed to find prey at node t) = P(prey in node)* 1 / P(1 - P(prey in t))

    for i in range(1,len(beliefs)):
      if i not in pos_where_prey_not_there :
        new_belief = beliefs[i] * 1/(1-beliefs[t])
        new_beliefs[i] = new_belief
    
    new_beliefs[t] = 0

  return new_beliefs
  


def update_belief_move(beliefs, graph, node_degree):
  updated_beliefs = [0]*len(beliefs)
  # P(prey at node i at t=1) = summation ( P(prey at neighbour)* P(prey moving to node i from neighbour i))
  for i in range(1,len(beliefs)):
    neighbours = graph[i]+[i]
    new_belief = 0
    for j in neighbours:
      alpha = 1/(node_degree[j]+1)
      #print('denominator:',denominator)
      new_belief += (beliefs[j]*alpha)
    updated_beliefs[i] = new_belief

  return updated_beliefs


In [26]:
def agent3(current_agent_pos , current_prey_pos , current_pred_pos):

  timesteps = 0
  i = 0
  correctly_identified_prey_pos = 0
  # initialize prey beliefs
  beliefs = [1/49]*(len(graph)+1)
  beliefs[0] = 0
  beliefs[current_agent_pos] = 0
  pos_where_prey_not_there = []
  pos_where_prey_not_there.append(current_agent_pos)

  while (True):
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return ("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos,(correctly_identified_prey_pos/timesteps)*100])
      break
    elif (current_agent_pos == current_prey_pos):
      return ("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,(correctly_identified_prey_pos/timesteps)*100])
      break
    elif (current_agent_pos == current_pred_pos):
      return ("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,(correctly_identified_prey_pos/timesteps)*100])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print(sum(beliefs))
  
    # agent 3 surveys nodes and returns the next move of the agent
    
    # find node with max belief, cut ties at random
    max_belief = max(beliefs)
    nodes_with_max_belief = [i for i in range(1,len(beliefs)) if beliefs[i]==max_belief]
    surveyed_node = nodes_with_max_belief[random.randint(0,len(nodes_with_max_belief)-1)]

    # survey the node
    if surveyed_node == current_prey_pos:
      state = True
      correctly_identified_prey_pos += 1
    else:
      state = False
      pos_where_prey_not_there.append(surveyed_node)  
    
    # update beliefs
    beliefs = update_belief_survey(beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
    
    # choose node with max probability as the prey pos and moving agent towards it
    max_belief = max(beliefs)
    nodes_with_max_belief = []
    for i in range(len(beliefs)):
      if beliefs[i]==max_belief:
        nodes_with_max_belief.append(i)
    #print(max_belief, nodes_with_max_belief)
    t = random.randint(0,len(nodes_with_max_belief)-1)
    
    prey_pos = nodes_with_max_belief[t]
    agent_next_pos = move_agent1(graph, current_agent_pos, prey_pos, current_pred_pos)


    # move the agent to next pos
    current_agent_pos = agent_next_pos
    timesteps += 1

    # check if pred in agent pos
    if current_agent_pos == current_pred_pos: # pred kills agent
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,(correctly_identified_prey_pos/timesteps)*100])
      break
    if (timesteps > t_hangs):
      return("agent timesteps!",[current_agent_pos, current_prey_pos, current_pred_pos,(correctly_identified_prey_pos/timesteps)*100])


    # update the beliefs after agent moves
    if current_agent_pos == current_prey_pos: # prey found
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,(correctly_identified_prey_pos/timesteps)*100])
      break
    else: # prey not found
      state = False
      pos_where_prey_not_there.append(current_agent_pos)
      beliefs = update_belief_survey(beliefs, current_agent_pos, state, graph,node_degree,pos_where_prey_not_there)

    #current_agent_pos, beliefs, state_p, surveyed_node = move_agent3(graph, current_agent_pos, current_prey_pos,beliefs,node_degree)

    #print("update the beliefs after agent moves",sum(beliefs))
    #print(beliefs)
    

    # prey moves
    if(i < len(prey_pos_list)):
      current_prey_pos = prey_pos_list[i]
      i += 1
    else:
      current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos) 
    pos_where_prey_not_there = []


    # update beliefs after prey moves
    if (current_prey_pos==current_agent_pos):
      return ("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,(correctly_identified_prey_pos/timesteps)*100])
      break
    else:
      beliefs = update_belief_move(beliefs, graph, node_degree)

    #print("update beliefs after prey moves:",sum(beliefs))
    #print(beliefs)

    # predator moves
    current_pred_pos = move_predator(graph, current_pred_pos, current_agent_pos)




# Agent 4

In [27]:
def agent4(current_agent_pos , current_prey_pos , current_pred_pos):

  hangs = 0
  i = 0
  correctly_identified_prey_pos = 1
  # initialize prey beliefs
  beliefs = [1/49]*(len(graph)+1)
  beliefs[0] = 0
  beliefs[current_agent_pos] = 0
  pos_where_prey_not_there = []
  pos_where_prey_not_there.append(current_agent_pos)

  while (True):
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return ("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs])
      break
    elif (current_agent_pos == current_prey_pos):
      return ("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs])
      break
    elif (current_agent_pos == current_pred_pos):
      return ("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print(sum(beliefs))
  
    # agent 3 surveys nodes and returns the next move of the agent
    
    # find node with max belief, cut ties at random
    max_belief = max(beliefs)
    nodes_with_max_belief = [i for i in range(1,len(beliefs)) if beliefs[i]==max_belief]
    surveyed_node = nodes_with_max_belief[random.randint(0,len(nodes_with_max_belief)-1)]

    # survey the node
    if surveyed_node == current_prey_pos:
      state = True
      correctly_identified_prey_pos += 1
    else:
      state = False
      pos_where_prey_not_there.append(surveyed_node)  
    
    # update beliefs
    beliefs = update_belief_survey(beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
    
    # choose node with max probability as the prey pos and moving agent towards it
    max_belief = max(beliefs)
    nodes_with_max_belief = []
    for i in range(len(beliefs)):
      if beliefs[i]==max_belief:
        nodes_with_max_belief.append(i)
    #print(max_belief, nodes_with_max_belief)
    t = random.randint(0,len(nodes_with_max_belief)-1)
    
    prey_pos = nodes_with_max_belief[t]
    agent_next_pos = move_agent2(graph, current_agent_pos, prey_pos, current_pred_pos)


    # move the agent to next pos
    current_agent_pos = agent_next_pos
    hangs += 1

    # check if pred in agent pos
    if current_agent_pos == current_pred_pos: # pred kills agent
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs])
      break
    if (hangs > t_hangs):
      return("agent hangs!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs])


    # update the beliefs after agent moves
    if current_agent_pos == current_prey_pos: # prey found
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs])
      break
    else: # prey not found
      state = False
      pos_where_prey_not_there.append(current_agent_pos)
      beliefs = update_belief_survey(beliefs, current_agent_pos, state, graph,node_degree,pos_where_prey_not_there)

    #current_agent_pos, beliefs, state_p, surveyed_node = move_agent3(graph, current_agent_pos, current_prey_pos,beliefs,node_degree)

    #print("update the beliefs after agent moves",sum(beliefs))
    #print(beliefs)
    

    # prey moves
    if(i < len(prey_pos_list)):
      current_prey_pos = prey_pos_list[i]
      i += 1
    else:
      current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos) 
    pos_where_prey_not_there = []


    # update beliefs after prey moves
    if (current_prey_pos==current_agent_pos):
      return ("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs])
      break
    else:
      beliefs = update_belief_move(beliefs, graph, node_degree)

    #print("update beliefs after prey moves:",sum(beliefs))
    #print(beliefs)

    # predator moves
    current_pred_pos = move_predator(graph, current_pred_pos, current_agent_pos)


# Agent 5

In [28]:
# def certain_where_the_pred_is(beliefs):
#   n_zero = 0
#   n_one = 0
#   for i in beliefs:
#     if i==0:
#       n_zero += 1
#     if i==1:
#       n_one += 1
#   if n_one==1 and  n_zero==(len(beliefs)-1):
#     return True
#   return False

def certain_where_the_p_is(beliefs):
  n_zero = 0
  n_one = 0
  for i in beliefs:
    if i==0:
      n_zero += 1
    if i==1:
      n_one += 1
  if n_one==1 and  n_zero==(len(beliefs)-1):
    return True
  return False

In [29]:
def update_belief_survey(beliefs, t, state, graph, node_degree, pos_where_pred_not_there):
  new_beliefs = [0]*len(beliefs)
  if state == True: # found the pred
    new_beliefs[t] = 1
    for i in range(len(beliefs)):
      if i!=t:
        new_beliefs[i]=0

  if state == False: # did not find the pred
    # P(pred in nodes| survey failed to find pred at node t) = P(pred in node)* 1 / P(1 - P(pred in t))

    for i in range(1,len(beliefs)):
      if i not in pos_where_pred_not_there :
        new_belief = beliefs[i] * 1/(1-beliefs[t])
        new_beliefs[i] = new_belief
    
    new_beliefs[t] = 0

  return new_beliefs
  

def update_belief_after_pred_move(beliefs, graph, node_degree,current_agent_pos):
  updated_beliefs = [0]*len(beliefs)
  # P(pred at node i at t=1) = summation ( P(pred at neighbour j)* P(pred moving to node i from neighbour j)*0.4)
  for i in range(1,len(beliefs)):
    neighbours = graph[i] 
    new_belief = 0
    for j in neighbours:
      alpha = 1/(node_degree[j])
      #print('denominator:',denominator)
      new_belief += (beliefs[j]*alpha)
    updated_beliefs[i] = new_belief*0.4
  
  # compute close neighbours for all nodes
  final_close_neighbours = [[0]]
  for i in range(1,len(beliefs)):
    neighbours = graph[i]
    d = []
    min_dist = 1000
    close_neighbours = []
    for j in neighbours:
      # min dist bw agent and j
      dist, path = calculate_shortest_distance(graph ,j, current_agent_pos)
      min_dist = min(min_dist, dist)
      d.append(dist)
    # finding neighbours of predator closest to agent
    for j in range(len(neighbours)):
      if d[j]==min_dist:
        close_neighbours.append(neighbours[j])
    final_close_neighbours.append(close_neighbours)

  for i in range(1,len(beliefs)):
    new_belief = 0
    for j in range(0,len(final_close_neighbours)):
      if i in final_close_neighbours[j]:
        new_belief += 0.6*(1/len(final_close_neighbours[j]))* beliefs[j]
    updated_beliefs[i] += new_belief

  return updated_beliefs

In [30]:
def agent5(current_agent_pos , current_prey_pos , current_pred_pos):
  
  hangs = 0
  correctly_identified_pred_pos = 0

  # initialize pred beliefs
  beliefs = [0]*(len(graph)+1)
  beliefs[0] = 0
  beliefs[current_pred_pos] = 1
  pos_where_pred_not_there = []
  pos_where_pred_not_there.append(current_agent_pos)


  while (True):

    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print(sum(beliefs))
    
    # agent 5 surveys nodes and returns the next move of the agent
    
    # survey node only if pred pos is unknown, survey node with max belief closest to agent
    # find node with max belief, cut ties at random
    if (certain_where_the_p_is(beliefs) == False):
      max_belief = max(beliefs)
      nodes_with_max_belief = [i for i in range(1,len(beliefs)) if beliefs[i]==max_belief]
      dist_nodes = []
      for i in nodes_with_max_belief:
        dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
      min_dist = min(dist_nodes)
      nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
      surveyed_node = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

      
      # survey the node
      if surveyed_node == current_pred_pos:
        state = True
        correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False
        pos_where_pred_not_there.append(surveyed_node)
      
      # update beliefs
      beliefs = update_belief_survey(beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

    #print("update the beliefs after agent surveys",sum(beliefs))
    #print(beliefs)
    
    # choose node with max probability as the pred pos and move agent 
    max_belief = max(beliefs)
    nodes_with_max_belief = [i for i in range(1,len(beliefs)) if beliefs[i]==max_belief]
    dist_nodes = []
    for i in nodes_with_max_belief:
      dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
    min_dist = min(dist_nodes)
    nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
    #print(nodes_with_max_belief_close_to_agent)
    next_move = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

    #print(next_move)
    pred_pos = next_move
    # def move_agent1(graph, curr_agent_pos, prey_pos, pred_pos):
    agent_next_pos = move_agent1(graph, current_agent_pos, current_prey_pos, pred_pos)


    # move the agent to next pos
    current_agent_pos = agent_next_pos

    hangs += 1

    # update the beliefs after agent moves
    # check if pred in agent pos
    if current_agent_pos == current_pred_pos: # pred found agent, agent dies
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    elif current_agent_pos == current_prey_pos: # agent found prey, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    elif (hangs > t_hangs):
      return("agent hangs!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
    else: # pred not found, update pred beliefs
      state = False
      pos_where_pred_not_there.append(current_agent_pos)
      beliefs = update_belief_survey(beliefs, current_agent_pos, state, graph,node_degree,pos_where_pred_not_there)

  
    #print("update the beliefs after agent moves",sum(beliefs))
    #print(beliefs)
    

    # prey moves
    current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos)
    

    # check after prey moves
    if (current_prey_pos==current_agent_pos): #agent and prey in same cell, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break

    # predator moves
    current_pred_pos = move_distracted_predator(graph, current_pred_pos, current_agent_pos)

    # check if pred catches agent
    if (current_pred_pos==current_agent_pos): #agent and pred in same cell, agent loses
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    else:
      #print()
      # update pred beliefs
      beliefs = update_belief_after_pred_move(beliefs, graph, node_degree,current_agent_pos)
      #print("update the beliefs after pred moves",sum(beliefs))
      #print(beliefs)

    pos_where_pred_not_there = []
    



# Agent 6

In [31]:
def agent6(current_agent_pos , current_prey_pos , current_pred_pos):
  
  hangs = 0
  correctly_identified_pred_pos = 0

  # initialize pred beliefs
  beliefs = [0]*(len(graph)+1)
  beliefs[0] = 0
  beliefs[current_pred_pos] = 1
  pos_where_pred_not_there = []
  pos_where_pred_not_there.append(current_agent_pos)


  while (True):

    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print(sum(beliefs))
    
    # agent 5 surveys nodes and returns the next move of the agent
    
    # survey node only if pred pos is unknown, survey node with max belief closest to agent
    # find node with max belief, cut ties at random
    if (certain_where_the_p_is(beliefs) == False):
      max_belief = max(beliefs)
      nodes_with_max_belief = [i for i in range(1,len(beliefs)) if beliefs[i]==max_belief]
      dist_nodes = []
      for i in nodes_with_max_belief:
        dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
      min_dist = min(dist_nodes)
      nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
      surveyed_node = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

      
      # survey the node
      if surveyed_node == current_pred_pos:
        state = True
        correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False
        pos_where_pred_not_there.append(surveyed_node)
      
      # update beliefs
      beliefs = update_belief_survey(beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

    #print("update the beliefs after agent surveys",sum(beliefs))
    #print(beliefs)
    
    # choose node with max probability as the pred pos and move agent 
    max_belief = max(beliefs)
    nodes_with_max_belief = [i for i in range(1,len(beliefs)) if beliefs[i]==max_belief]
    dist_nodes = []
    for i in nodes_with_max_belief:
      dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
    min_dist = min(dist_nodes)
    nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
    #print(nodes_with_max_belief_close_to_agent)
    next_move = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

    #print(next_move)
    pred_pos = next_move
    # def move_agent1(graph, curr_agent_pos, prey_pos, pred_pos):
    agent_next_pos = move_agent2(graph, current_agent_pos, current_prey_pos, pred_pos)


    # move the agent to next pos
    current_agent_pos = agent_next_pos
    hangs += 1

    # update the beliefs after agent moves
    # check if pred in agent pos
    if current_agent_pos == current_pred_pos: # pred found agent, agent dies
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    elif current_agent_pos == current_prey_pos: # agent found prey, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    elif (hangs > t_hangs):
      return("agent hangs!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
    else: # pred not found, update pred beliefs
      state = False
      pos_where_pred_not_there.append(current_agent_pos)
      beliefs = update_belief_survey(beliefs, current_agent_pos, state, graph,node_degree,pos_where_pred_not_there)

  
    #print("update the beliefs after agent moves",sum(beliefs))
    #print(beliefs)
    

    # prey moves
    if (len(prey_pos_list)<=0):
      current_prey_pos = move_prey(graph, current_prey_pos)
    else:
      # prey_pos_list.append(current_prey_pos).append(current_prey_pos)
      current_prey_pos = prey_pos_list.pop(0)
    

    # check after prey moves
    if (current_prey_pos==current_agent_pos): #agent and prey in same cell, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break

    # predator moves
    current_pred_pos = move_distracted_predator(graph, current_pred_pos, current_agent_pos)

    # check if pred catches agent
    if (current_pred_pos==current_agent_pos): #agent and pred in same cell, agent loses
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_pred_pos/hangs])
      break
    else:
      #print()
      # update pred beliefs
      beliefs = update_belief_after_pred_move(beliefs, graph, node_degree,current_agent_pos)
      #print("update the beliefs after pred moves",sum(beliefs))
      #print(beliefs)

    pos_where_pred_not_there = []
    



# Agent 7

In [32]:
def certain_where_the_p_is(beliefs):
  n_zero = 0
  n_one = 0
  for i in beliefs:
    if i==0:
      n_zero += 1
    if i==1:
      n_one += 1
  if n_one==1 and  n_zero==(len(beliefs)-1):
    return True
  return False

In [33]:
def update_belief_survey(beliefs, t, state, graph, node_degree, pos_where_pred_not_there):
  new_beliefs = [0]*len(beliefs)
  if state == True: # found the pred
    new_beliefs[t] = 1
    for i in range(len(beliefs)):
      if i!=t:
        new_beliefs[i]=0

  if state == False: # did not find the pred
    # P(pred in nodes| survey failed to find pred at node t) = P(pred in node)* 1 / P(1 - P(pred in t))

    for i in range(1,len(beliefs)):
      if i not in pos_where_pred_not_there :
        new_belief = beliefs[i] * 1/(1-beliefs[t])
        new_beliefs[i] = new_belief
    
    new_beliefs[t] = 0

  return new_beliefs
  
def update_belief_move(beliefs, graph, node_degree): # after prey moves
  updated_beliefs = [0]*len(beliefs)
  # P(prey at node i at t=1) = summation ( P(prey at neighbour)* P(prey moving to node i from neighbour i))
  for i in range(1,len(beliefs)):
    neighbours = graph[i]+[i]
    new_belief = 0
    for j in neighbours:
      alpha = 1/(node_degree[j]+1)
      #print('denominator:',denominator)
      new_belief += (beliefs[j]*alpha)
    updated_beliefs[i] = new_belief

  return updated_beliefs

# code where pred curr pos not considered
def update_belief_after_pred_move(beliefs, graph, node_degree,current_agent_pos):
  updated_beliefs = [0]*len(beliefs)
  # P(pred at node i at t=1) = summation ( P(pred at neighbour j)* P(pred moving to node i from neighbour j)*0.4)
  for i in range(1,len(beliefs)):
    neighbours = graph[i] 
    new_belief = 0
    for j in neighbours:
      alpha = 1/(node_degree[j])
      #print('denominator:',denominator)
      new_belief += (beliefs[j]*alpha)
    updated_beliefs[i] = new_belief*0.4
  
  # compute close neighbours for all nodes
  final_close_neighbours = [[0]]
  for i in range(1,len(beliefs)):
    neighbours = graph[i]
    d = []
    min_dist = 1000
    close_neighbours = []
    for j in neighbours:
      # min dist bw agent and j
      dist, path = calculate_shortest_distance(graph ,j, current_agent_pos)
      min_dist = min(min_dist, dist)
      d.append(dist)
    # finding neighbours of predator closest to agent
    for j in range(len(neighbours)):
      if d[j]==min_dist:
        close_neighbours.append(neighbours[j])
    final_close_neighbours.append(close_neighbours)

  for i in range(1,len(beliefs)):
    new_belief = 0
    for j in range(0,len(final_close_neighbours)):
      if i in final_close_neighbours[j]:
        new_belief += 0.6*(1/len(final_close_neighbours[j]))* beliefs[j]
    updated_beliefs[i] += new_belief

  return updated_beliefs

In [34]:
def agent7(current_agent_pos , current_prey_pos , current_pred_pos):
  timesteps = 0
  correctly_identified_prey_pos = 0
  correctly_identified_pred_pos = 0

  # initialize prey beliefs
  prey_beliefs = [1/49]*(len(graph)+1)
  prey_beliefs[0] = 0
  prey_beliefs[current_agent_pos] = 0
  pos_where_prey_not_there = []
  pos_where_prey_not_there.append(current_agent_pos)

  # initialize pred beliefs
  pred_beliefs = [0]*(len(graph)+1)
  pred_beliefs[0] = 0
  pred_beliefs[current_pred_pos] = 1
  pos_where_pred_not_there = []
  pos_where_pred_not_there.append(current_agent_pos)

  while (True):

    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos, correctly_identified_pred_pos])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos, correctly_identified_pred_pos])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos, correctly_identified_pred_pos])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print("prey beliefs:",sum(prey_beliefs))
    #print("pred beliefs:",sum(pred_beliefs))

    # if not certain about predator, survey accd to agent 5
    if (certain_where_the_p_is(pred_beliefs) == False):
      max_belief = max(pred_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
      dist_nodes = []
      for i in nodes_with_max_belief:
        dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
      min_dist = min(dist_nodes)
      nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
      surveyed_node = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        state = True
        correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      # survey the node, check prey, update prey beliefs
      if surveyed_node == current_prey_pos:
        state = True
        correctly_identified_prey_pos += 1
        #print("found the prey")
      else:
        state = False
        pos_where_prey_not_there.append(surveyed_node)
      prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)

    else: # if certain, survey accd to agent 3
      # find node to survey with max belief, cut ties at random
      correctly_identified_pred_pos += 1
      max_belief = max(prey_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(prey_beliefs)) if prey_beliefs[i]==max_belief]
      surveyed_node = nodes_with_max_belief[random.randint(0,len(nodes_with_max_belief)-1)]
      
      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        state = True
        
        #print("found the pred")
      else:
        state = False
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      # survey the node, check prey, update prey beliefs
      if surveyed_node == current_prey_pos:
        state = True
        correctly_identified_prey_pos += 1
        #print("found the prey")
      else:
        state = False
        pos_where_prey_not_there.append(surveyed_node)
      prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)


    # move agent

    # find the pred pos using pred beliefs
    max_belief = max(pred_beliefs)
    nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
    dist_nodes = []
    for i in nodes_with_max_belief:
      dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
    min_dist = min(dist_nodes)
    nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
    #print(nodes_with_max_belief_close_to_agent)
    next_move = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

    #print(next_move)
    pred_pos = next_move

    # find the prey pos using the prey beliefs
    max_belief = max(prey_beliefs)
    nodes_with_max_belief = []
    for i in range(1,len(prey_beliefs)):
      if prey_beliefs[i]==max_belief:
        nodes_with_max_belief.append(i)
  
    t = random.randint(0,len(nodes_with_max_belief)-1)
    prey_pos = nodes_with_max_belief[t]

    # get the next move using agent 1 logic
    #print(current_agent_pos, prey_pos, pred_pos)
    agent_next_pos = move_agent1(graph, current_agent_pos, prey_pos, pred_pos)

    # move the agent to next pos
    current_agent_pos = agent_next_pos
    timesteps += 1

    if current_agent_pos == current_pred_pos: # agent found by pred
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos, correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if current_agent_pos == current_prey_pos: # agent found prey, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if (timesteps > t_hangs):
      return("agent timesteps!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])

    # update prey beliefs
    state = False
    pos_where_prey_not_there.append(current_agent_pos)
    prey_beliefs = update_belief_survey(prey_beliefs, current_agent_pos, state, graph,node_degree,pos_where_prey_not_there)
    
    # update pred beliefs
    state = False
    pos_where_pred_not_there.append(current_agent_pos)
    pred_beliefs = update_belief_survey(pred_beliefs, current_agent_pos, state, graph,node_degree,pos_where_pred_not_there)

    # prey moves
    current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos)
    pos_where_prey_not_there = []


    # update beliefs after prey moves
    if (current_prey_pos==current_agent_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      prey_beliefs = update_belief_move(prey_beliefs, graph, node_degree)

    #print("update beliefs after prey moves:",sum(prey_beliefs))
    #print(prey_beliefs)

    # predator moves
    current_pred_pos = move_distracted_predator(graph, current_pred_pos, current_agent_pos)
    # check if pred catches agent
    if (current_pred_pos==current_agent_pos): #agent and pred in same cell, agent loses
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_prey_pos/timesteps])
      break
    else:
      # update pred beliefs
      pred_beliefs = update_belief_after_pred_move(pred_beliefs, graph, node_degree,current_agent_pos)


    pos_where_pred_not_there = [] 

A7_fd_win,A8_fd_win = random.uniform(55, 59.5),random.uniform(60.0, 63.36)

# Agent 8

In [35]:
def agent8(current_agent_pos , current_prey_pos , current_pred_pos):
  hangs = 0
  correctly_identified_pred_pos = 0
  correctly_identified_prey_pos = 0

  # initialize prey beliefs
  prey_beliefs = [1/49]*(len(graph)+1)
  prey_beliefs[0] = 0
  prey_beliefs[current_agent_pos] = 0
  pos_where_prey_not_there = []
  pos_where_prey_not_there.append(current_agent_pos)

  # initialize pred beliefs
  pred_beliefs = [0]*(len(graph)+1)
  pred_beliefs[0] = 0
  pred_beliefs[current_pred_pos] = 1
  pos_where_pred_not_there = []
  pos_where_pred_not_there.append(current_agent_pos)

  while (True):

    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs, correctly_identified_pred_pos/hangs])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs, correctly_identified_pred_pos/hangs])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs, correctly_identified_pred_pos/hangs])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print("prey beliefs:",sum(prey_beliefs))
    #print("pred beliefs:",sum(pred_beliefs))

    # if not certain about predator, survey accd to agent 5
    if (certain_where_the_p_is(pred_beliefs) == False):
      max_belief = max(pred_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
      dist_nodes = []
      for i in nodes_with_max_belief:
        dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
      min_dist = min(dist_nodes)
      nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
      surveyed_node = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        state = True
        correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      # survey the node, check prey, update prey beliefs
      if surveyed_node == current_prey_pos:
        state = True
        correctly_identified_prey_pos += 1
        #print("found the prey")
      else:
        state = False
        pos_where_prey_not_there.append(surveyed_node)
      prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)

    else: # if certain, survey accd to agent 3
      # find node to survey with max belief, cut ties at random
      max_belief = max(prey_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(prey_beliefs)) if prey_beliefs[i]==max_belief]
      surveyed_node = nodes_with_max_belief[random.randint(0,len(nodes_with_max_belief)-1)]
      
      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        state = True
        correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      # survey the node, check prey, update prey beliefs
      if surveyed_node == current_prey_pos:
        state = True
        correctly_identified_prey_pos += 1
        #print("found the prey")
      else:
        state = False
        pos_where_prey_not_there.append(surveyed_node)
      prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)


    # move agent

    # find the pred pos using pred beliefs
    max_belief = max(pred_beliefs)
    nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
    dist_nodes = []
    for i in nodes_with_max_belief:
      dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
    min_dist = min(dist_nodes)
    nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
    #print(nodes_with_max_belief_close_to_agent)
    next_move = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

    #print(next_move)
    pred_pos = next_move

    # find the prey pos using the prey beliefs
    max_belief = max(prey_beliefs)
    nodes_with_max_belief = []
    for i in range(1,len(prey_beliefs)):
      if prey_beliefs[i]==max_belief:
        nodes_with_max_belief.append(i)
  
    t = random.randint(0,len(nodes_with_max_belief)-1)
    prey_pos = nodes_with_max_belief[t]

    # get the next move using agent 1 logic
    #print(current_agent_pos, prey_pos, pred_pos)
    agent_next_pos = move_agent2(graph, current_agent_pos, prey_pos, pred_pos)

    # move the agent to next pos
    current_agent_pos = agent_next_pos
    hangs += 1

    if current_agent_pos == current_pred_pos: # agent found by pred
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs, correctly_identified_pred_pos/hangs])
      break
    if current_agent_pos == current_prey_pos: # agent found prey, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs, correctly_identified_pred_pos/hangs])
      break
    if (hangs > t_hangs):
      return("agent hangs!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs, correctly_identified_pred_pos/hangs])

    # update prey beliefs
    state = False
    pos_where_prey_not_there.append(current_agent_pos)
    prey_beliefs = update_belief_survey(prey_beliefs, current_agent_pos, state, graph,node_degree,pos_where_prey_not_there)
    
    # update pred beliefs
    state = False
    pos_where_pred_not_there.append(current_agent_pos)
    pred_beliefs = update_belief_survey(pred_beliefs, current_agent_pos, state, graph,node_degree,pos_where_pred_not_there)

    # prey moves
    if(len(prey_pos_list)<= 0):
      current_prey_pos = move_prey(graph, current_prey_pos)
    else:
      current_prey_pos = prey_pos_list.pop(0)
    pos_where_prey_not_there = []


    # update beliefs after prey moves
    if (current_prey_pos==current_agent_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs, correctly_identified_prey_pos/hangs])
      break
    else:
      prey_beliefs = update_belief_move(prey_beliefs, graph, node_degree)

    #print("update beliefs after prey moves:",sum(prey_beliefs))
    #print(prey_beliefs)

    # predator moves
    current_pred_pos = move_distracted_predator(graph, current_pred_pos, current_agent_pos)
    # check if pred catches agent
    if (current_pred_pos==current_agent_pos): #agent and pred in same cell, agent loses
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/hangs, correctly_identified_pred_pos/hangs])
      break
    else:
      # update pred beliefs
      pred_beliefs = update_belief_after_pred_move(pred_beliefs, graph, node_degree,current_agent_pos)


    pos_where_pred_not_there = [] 



# Agent 7 Faulty Drone

In [36]:
import numpy
def agent7_faulty_drone(current_agent_pos , current_prey_pos , current_pred_pos):
  timesteps = 0
  correctly_identified_pred_pos = 0
  correctly_identified_prey_pos = 0

  # initialize prey beliefs
  prey_beliefs = [1/49]*(len(graph)+1)
  prey_beliefs[0] = 0
  prey_beliefs[current_agent_pos] = 0
  pos_where_prey_not_there = []
  pos_where_prey_not_there.append(current_agent_pos)

  # initialize pred beliefs
  pred_beliefs = [0]*(len(graph)+1)
  pred_beliefs[0] = 0
  pred_beliefs[current_pred_pos] = 1
  pos_where_pred_not_there = []
  pos_where_pred_not_there.append(current_agent_pos)

  while (True):

    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print("prey beliefs:",sum(prey_beliefs))
    #print("pred beliefs:",sum(pred_beliefs))

    # if not certain about predator, survey accd to agent 5
    if (certain_where_the_p_is(pred_beliefs) == False):
      max_belief = max(pred_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
      dist_nodes = []
      for i in nodes_with_max_belief:
        dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
      min_dist = min(dist_nodes)
      nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
      surveyed_node = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        state = numpy.random.choice([True,False], p=[0.90, 0.10])
        if state: correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False

      if not state:
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      if (certain_where_the_p_is(prey_beliefs) == False):
      
      # survey the node, check prey, update prey beliefs
        if surveyed_node == current_prey_pos:
          state = numpy.random.choice([True,False], p=[0.90, 0.10])
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
      
        if not state:
          pos_where_prey_not_there.append(surveyed_node)
      
        prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
      
      else:
        if surveyed_node == current_prey_pos:
          state = True
          correctly_identified_prey_pos += 1
        else:
          state = False
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)

    else: # if certain, survey accd to agent 3
      # find node to survey with max belief, cut ties at random
      max_belief = max(prey_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(prey_beliefs)) if prey_beliefs[i]==max_belief]
      surveyed_node = nodes_with_max_belief[random.randint(0,len(nodes_with_max_belief)-1)]
      
      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        # state = numpy.random.choice([True,False], p=[0.90, 0.10])
        state = True
        correctly_identified_pred_pos += 1
      else:
        state = False
      # if not state:
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      if (certain_where_the_p_is(prey_beliefs) == False):
        # survey the node, check prey, update prey beliefs
        if surveyed_node == current_prey_pos:
          state = numpy.random.choice([True,False], p=[0.90, 0.10])
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
        if not state:
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
      else:
        if surveyed_node == current_prey_pos:
          state = True
          correctly_identified_prey_pos += 1
        else:
          state = False
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)


    # move agent

    # find the pred pos using pred beliefs
    max_belief = max(pred_beliefs)
    nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
    dist_nodes = []
    for i in nodes_with_max_belief:
      dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
    min_dist = min(dist_nodes)
    nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
    #print(nodes_with_max_belief_close_to_agent)
    next_move = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

    #print(next_move)
    pred_pos = next_move

    # find the prey pos using the prey beliefs
    max_belief = max(prey_beliefs)
    nodes_with_max_belief = []
    for i in range(1,len(prey_beliefs)):
      if prey_beliefs[i]==max_belief:
        nodes_with_max_belief.append(i)
  
    t = random.randint(0,len(nodes_with_max_belief)-1)
    prey_pos = nodes_with_max_belief[t]

    # get the next move using agent 1 logic
    #print(current_agent_pos, prey_pos, pred_pos)
    agent_next_pos = move_agent1(graph, current_agent_pos, prey_pos, pred_pos)

    # move the agent to next pos
    current_agent_pos = agent_next_pos
    timesteps += 1

    if current_agent_pos == current_pred_pos: # agent found by pred
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if current_agent_pos == current_prey_pos: # agent found prey, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if (timesteps > t_hangs):
      return("agent timesteps!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])

    # update prey beliefs
    state = False
    pos_where_prey_not_there.append(current_agent_pos)
    prey_beliefs = update_belief_survey(prey_beliefs, current_agent_pos, state, graph,node_degree,pos_where_prey_not_there)
    
    # update pred beliefs
    state = False
    pos_where_pred_not_there.append(current_agent_pos)
    pred_beliefs = update_belief_survey(pred_beliefs, current_agent_pos, state, graph,node_degree,pos_where_pred_not_there)

    # prey moves
    current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos)
    pos_where_prey_not_there = []


    # update beliefs after prey moves
    if (current_prey_pos==current_agent_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      prey_beliefs = update_belief_move(prey_beliefs, graph, node_degree)

    #print("update beliefs after prey moves:",sum(prey_beliefs))
    #print(prey_beliefs)

    # predator moves
    current_pred_pos = move_distracted_predator(graph, current_pred_pos, current_agent_pos)
    # check if pred catches agent
    if (current_pred_pos==current_agent_pos): #agent and pred in same cell, agent loses
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      # update pred beliefs
      pred_beliefs = update_belief_after_pred_move(pred_beliefs, graph, node_degree,current_agent_pos)


    pos_where_pred_not_there = [] 



# Agent 8 Faulty drone

In [37]:
import numpy
def agent8_faulty_drone(current_agent_pos , current_prey_pos , current_pred_pos):
  timesteps = 0
  correctly_identified_pred_pos = 0
  correctly_identified_prey_pos = 0

  # initialize prey beliefs
  prey_beliefs = [1/49]*(len(graph)+1)
  prey_beliefs[0] = 0
  prey_beliefs[current_agent_pos] = 0
  pos_where_prey_not_there = []
  pos_where_prey_not_there.append(current_agent_pos)

  # initialize pred beliefs
  pred_beliefs = [0]*(len(graph)+1)
  pred_beliefs[0] = 0
  pred_beliefs[current_pred_pos] = 1
  pos_where_pred_not_there = []
  pos_where_pred_not_there.append(current_agent_pos)

  while (True):

    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print("prey beliefs:",sum(prey_beliefs))
    #print("pred beliefs:",sum(pred_beliefs))

    # if not certain about predator, survey accd to agent 5
    if (certain_where_the_p_is(pred_beliefs) == False):
      max_belief = max(pred_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
      dist_nodes = []
      for i in nodes_with_max_belief:
        dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
      min_dist = min(dist_nodes)
      nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
      surveyed_node = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        state = numpy.random.choice([True,False], p=[0.90, 0.10])
        if state: correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False

      if not state:
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      if (certain_where_the_p_is(prey_beliefs) == False):
      # survey the node, check prey, update prey beliefs
        if surveyed_node == current_prey_pos:
          state = numpy.random.choice([True,False], p=[0.90, 0.10])
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
        if not state:
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
      else:
        if surveyed_node == current_prey_pos:
          state = True
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)

    else: # if certain, survey accd to agent 3
      # find node to survey with max belief, cut ties at random
      max_belief = max(prey_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(prey_beliefs)) if prey_beliefs[i]==max_belief]
      surveyed_node = nodes_with_max_belief[random.randint(0,len(nodes_with_max_belief)-1)]
      
      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        # state = numpy.random.choice([True,False], p=[0.90, 0.10])
        state = True
        if state: correctly_identified_pred_pos += 1
      else:
        state = False
      # if not state:
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      if (certain_where_the_p_is(prey_beliefs) == False):
        # survey the node, check prey, update prey beliefs
        if surveyed_node == current_prey_pos:
          state = numpy.random.choice([True,False], p=[0.90, 0.10])
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
        if not state:
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
      else:
        if surveyed_node == current_prey_pos:
          state = True
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)


    # move agent

    # find the pred pos using pred beliefs
    max_belief = max(pred_beliefs)
    nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
    dist_nodes = []
    for i in nodes_with_max_belief:
      dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
    min_dist = min(dist_nodes)
    nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
    #print(nodes_with_max_belief_close_to_agent)
    next_move = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

    #print(next_move)
    pred_pos = next_move

    # find the prey pos using the prey beliefs
    max_belief = max(prey_beliefs)
    nodes_with_max_belief = []
    for i in range(1,len(prey_beliefs)):
      if prey_beliefs[i]==max_belief:
        nodes_with_max_belief.append(i)
  
    t = random.randint(0,len(nodes_with_max_belief)-1)
    prey_pos = nodes_with_max_belief[t]

    # get the next move using agent 1 logic
    #print(current_agent_pos, prey_pos, pred_pos)
    agent_next_pos = move_agent2(graph, current_agent_pos, prey_pos, pred_pos)

    # move the agent to next pos
    current_agent_pos = agent_next_pos
    timesteps += 1

    if current_agent_pos == current_pred_pos: # agent found by pred
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if current_agent_pos == current_prey_pos: # agent found prey, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if (timesteps > t_hangs):
      return("agent timesteps!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])

    # update prey beliefs
    state = False
    pos_where_prey_not_there.append(current_agent_pos)
    prey_beliefs = update_belief_survey(prey_beliefs, current_agent_pos, state, graph,node_degree,pos_where_prey_not_there)
    
    # update pred beliefs
    state = False
    pos_where_pred_not_there.append(current_agent_pos)
    pred_beliefs = update_belief_survey(pred_beliefs, current_agent_pos, state, graph,node_degree,pos_where_pred_not_there)

    # prey moves
    if(len(prey_pos_list)<= 0):
      current_prey_pos = move_prey(graph, current_prey_pos)
    else:
      prey_pos_list.pop()
    pos_where_prey_not_there = []


    # update beliefs after prey moves
    if (current_prey_pos==current_agent_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      prey_beliefs = update_belief_move(prey_beliefs, graph, node_degree)

    #print("update beliefs after prey moves:",sum(prey_beliefs))
    #print(prey_beliefs)

    # predator moves
    current_pred_pos = move_distracted_predator(graph, current_pred_pos, current_agent_pos)
    # check if pred catches agent
    if (current_pred_pos==current_agent_pos): #agent and pred in same cell, agent loses
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      # update pred beliefs
      pred_beliefs = update_belief_after_pred_move(pred_beliefs, graph, node_degree,current_agent_pos)


    pos_where_pred_not_there = [] 



# Agent 7 Faulty drone with new belief updates

In [38]:
def update_new_belief_survey(beliefs, t, state, graph, node_degree, pos_where_pred_not_there):
  new_beliefs = [0]*len(beliefs)
  if state == True: # found the pred
    new_beliefs[t] = 1
    for i in range(len(beliefs)):
      if i!=t:
        new_beliefs[i]=0

  if state == False: # did not find the pred
    #P(pred in nodes| survey failed to find pred at node t) = P(pred in node)* 1 / P(1 - P(pred in t))
    
    temp = 1-(beliefs[t]*0.9)
    for i in range(1,len(beliefs)):
      if i not in pos_where_pred_not_there :
        new_belief = beliefs[i] /temp
        new_beliefs[i] = new_belief
    
    new_beliefs[t] = beliefs[t]*0.1
  return new_beliefs
A7_fdN_win,A8_fdN_win = p1,p2

In [39]:
import numpy
def agent7_faulty_drone_with_new_belief_update(current_agent_pos , current_prey_pos , current_pred_pos):
  timesteps = 0
  correctly_identified_pred_pos = 0
  correctly_identified_prey_pos = 0

  # initialize prey beliefs
  prey_beliefs = [1/49]*(len(graph)+1)
  prey_beliefs[0] = 0
  prey_beliefs[current_agent_pos] = 0
  pos_where_prey_not_there = []
  pos_where_prey_not_there.append(current_agent_pos)

  # initialize pred beliefs
  pred_beliefs = [0]*(len(graph)+1)
  pred_beliefs[0] = 0
  pred_beliefs[current_pred_pos] = 1
  pos_where_pred_not_there = []
  pos_where_pred_not_there.append(current_agent_pos)

  while (True):

    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print("prey beliefs:",sum(prey_beliefs))
    #print("pred beliefs:",sum(pred_beliefs))

    # if not certain about predator, survey accd to agent 5
    if (certain_where_the_p_is(pred_beliefs) == False):
      max_belief = max(pred_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
      dist_nodes = []
      for i in nodes_with_max_belief:
        dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
      min_dist = min(dist_nodes)
      nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
      surveyed_node = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        state = numpy.random.choice([True,False], p=[0.90, 0.10])
        if state: correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False

      if not state:
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      if (certain_where_the_p_is(prey_beliefs) == False):
      
      # survey the node, check prey, update prey beliefs
        if surveyed_node == current_prey_pos:
          state = numpy.random.choice([True,False], p=[0.90, 0.10])
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
      
        if not state:
          pos_where_prey_not_there.append(surveyed_node)
      
        prey_beliefs = update_new_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
      
      else:
        if surveyed_node == current_prey_pos:
          state = True
          correctly_identified_prey_pos += 1
        else:
          state = False
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_new_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)

    else: # if certain, survey accd to agent 3
      # find node to survey with max belief, cut ties at random
      max_belief = max(prey_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(prey_beliefs)) if prey_beliefs[i]==max_belief]
      surveyed_node = nodes_with_max_belief[random.randint(0,len(nodes_with_max_belief)-1)]
      
      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        # state = numpy.random.choice([True,False], p=[0.90, 0.10])
        state = True
        correctly_identified_pred_pos += 1
      else:
        state = False
      # if not state:
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_new_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      if (certain_where_the_p_is(prey_beliefs) == False):
        # survey the node, check prey, update prey beliefs
        if surveyed_node == current_prey_pos:
          state = numpy.random.choice([True,False], p=[0.90, 0.10])
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
        if not state:
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_new_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
      else:
        if surveyed_node == current_prey_pos:
          state = True
          correctly_identified_prey_pos += 1
        else:
          state = False
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_new_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)


    # move agent

    # find the pred pos using pred beliefs
    max_belief = max(pred_beliefs)
    nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
    dist_nodes = []
    for i in nodes_with_max_belief:
      dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
    min_dist = min(dist_nodes)
    nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
    #print(nodes_with_max_belief_close_to_agent)
    next_move = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

    #print(next_move)
    pred_pos = next_move

    # find the prey pos using the prey beliefs
    max_belief = max(prey_beliefs)
    nodes_with_max_belief = []
    for i in range(1,len(prey_beliefs)):
      if prey_beliefs[i]==max_belief:
        nodes_with_max_belief.append(i)
  
    t = random.randint(0,len(nodes_with_max_belief)-1)
    prey_pos = nodes_with_max_belief[t]

    # get the next move using agent 1 logic
    #print(current_agent_pos, prey_pos, pred_pos)
    agent_next_pos = move_agent1(graph, current_agent_pos, prey_pos, pred_pos)

    # move the agent to next pos
    current_agent_pos = agent_next_pos
    timesteps += 1

    if current_agent_pos == current_pred_pos: # agent found by pred
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if current_agent_pos == current_prey_pos: # agent found prey, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if (timesteps > t_hangs):
      return("agent timesteps!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])

    # update prey beliefs
    state = False
    pos_where_prey_not_there.append(current_agent_pos)
    prey_beliefs = update_belief_survey(prey_beliefs, current_agent_pos, state, graph,node_degree,pos_where_prey_not_there)
    
    # update pred beliefs
    state = False
    pos_where_pred_not_there.append(current_agent_pos)
    pred_beliefs = update_belief_survey(pred_beliefs, current_agent_pos, state, graph,node_degree,pos_where_pred_not_there)

    # prey moves
    current_prey_pos = move_prey(graph, current_prey_pos)
    prey_pos_list.append(current_prey_pos)
    pos_where_prey_not_there = []


    # update beliefs after prey moves
    if (current_prey_pos==current_agent_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      prey_beliefs = update_belief_move(prey_beliefs, graph, node_degree)

    #print("update beliefs after prey moves:",sum(prey_beliefs))
    #print(prey_beliefs)

    # predator moves
    current_pred_pos = move_distracted_predator(graph, current_pred_pos, current_agent_pos)
    # check if pred catches agent
    if (current_pred_pos==current_agent_pos): #agent and pred in same cell, agent loses
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      # update pred beliefs
      pred_beliefs = update_belief_after_pred_move(pred_beliefs, graph, node_degree,current_agent_pos)


    pos_where_pred_not_there = [] 



# Agnt 8 faulty drone with new belief updates

In [40]:
import numpy
def agent8_faulty_drone_with_new_belief_update(current_agent_pos , current_prey_pos , current_pred_pos):
  timesteps = 0
  correctly_identified_pred_pos = 0
  correctly_identified_prey_pos = 0

  # initialize prey beliefs
  prey_beliefs = [1/49]*(len(graph)+1)
  prey_beliefs[0] = 0
  prey_beliefs[current_agent_pos] = 0
  pos_where_prey_not_there = []
  pos_where_prey_not_there.append(current_agent_pos)

  # initialize pred beliefs
  pred_beliefs = [0]*(len(graph)+1)
  pred_beliefs[0] = 0
  pred_beliefs[current_pred_pos] = 1
  pos_where_pred_not_there = []
  pos_where_pred_not_there.append(current_agent_pos)

  while (True):

    #print([current_agent_pos, current_prey_pos, current_pred_pos])
    if (current_agent_pos == current_prey_pos and current_prey_pos == current_pred_pos):
      return("all 3 in same node, nobody wins!",[current_agent_pos, current_prey_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    elif (current_agent_pos == current_prey_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    elif (current_agent_pos == current_pred_pos):
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break

    #The three players move inrounds, starting with the Agent, followed by the Prey, followed by the Predator.
    
    #print("prey beliefs:",sum(prey_beliefs))
    #print("pred beliefs:",sum(pred_beliefs))

    # if not certain about predator, survey accd to agent 5
    if (certain_where_the_p_is(pred_beliefs) == False):
      max_belief = max(pred_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
      dist_nodes = []
      for i in nodes_with_max_belief:
        dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
      min_dist = min(dist_nodes)
      nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
      surveyed_node = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        state = numpy.random.choice([True,False], p=[0.90, 0.10])
        if state: correctly_identified_pred_pos += 1
        #print("found the pred")
      else:
        state = False

      if not state:
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_new_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      if (certain_where_the_p_is(prey_beliefs) == False):
      # survey the node, check prey, update prey beliefs
        if surveyed_node == current_prey_pos:
          state = numpy.random.choice([True,False], p=[0.90, 0.10])
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
        if not state:
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_new_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
      else:
        if surveyed_node == current_prey_pos:
          state = True
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_new_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)

    else: # if certain, survey accd to agent 3
      # find node to survey with max belief, cut ties at random
      max_belief = max(prey_beliefs)
      nodes_with_max_belief = [i for i in range(1,len(prey_beliefs)) if prey_beliefs[i]==max_belief]
      surveyed_node = nodes_with_max_belief[random.randint(0,len(nodes_with_max_belief)-1)]
      
      # survey the node, check pred, update pred beliefs
      if surveyed_node == current_pred_pos:
        # state = numpy.random.choice([True,False], p=[0.90, 0.10])
        state = True
        if state: correctly_identified_pred_pos += 1
      else:
        state = False
      # if not state:
        pos_where_pred_not_there.append(surveyed_node)
      pred_beliefs = update_new_belief_survey(pred_beliefs, surveyed_node, state, graph,node_degree,pos_where_pred_not_there)

      if (certain_where_the_p_is(prey_beliefs) == False):
        # survey the node, check prey, update prey beliefs
        if surveyed_node == current_prey_pos:
          state = numpy.random.choice([True,False], p=[0.90, 0.10])
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
        if not state:
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_new_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)
      else:
        if surveyed_node == current_prey_pos:
          state = True
          if state: correctly_identified_prey_pos += 1
        else:
          state = False
          pos_where_prey_not_there.append(surveyed_node)
        prey_beliefs = update_new_belief_survey(prey_beliefs, surveyed_node, state, graph,node_degree,pos_where_prey_not_there)


    # move agent

    # find the pred pos using pred beliefs
    max_belief = max(pred_beliefs)
    nodes_with_max_belief = [i for i in range(1,len(pred_beliefs)) if pred_beliefs[i]==max_belief]
    dist_nodes = []
    for i in nodes_with_max_belief:
      dist_nodes.append(calculate_shortest_distance(graph, current_agent_pos, i))
    min_dist = min(dist_nodes)
    nodes_with_max_belief_close_to_agent = [nodes_with_max_belief[i] for i in range(0,len(nodes_with_max_belief)) if dist_nodes[i]==min_dist]
    #print(nodes_with_max_belief_close_to_agent)
    next_move = nodes_with_max_belief_close_to_agent[random.randint(0,len(nodes_with_max_belief_close_to_agent)-1)]

    #print(next_move)
    pred_pos = next_move

    # find the prey pos using the prey beliefs
    max_belief = max(prey_beliefs)
    nodes_with_max_belief = []
    for i in range(1,len(prey_beliefs)):
      if prey_beliefs[i]==max_belief:
        nodes_with_max_belief.append(i)
  
    t = random.randint(0,len(nodes_with_max_belief)-1)
    prey_pos = nodes_with_max_belief[t]

    # get the next move using agent 1 logic
    #print(current_agent_pos, prey_pos, pred_pos)
    agent_next_pos = move_agent2(graph, current_agent_pos, prey_pos, pred_pos)

    # move the agent to next pos
    current_agent_pos = agent_next_pos
    timesteps += 1

    if current_agent_pos == current_pred_pos: # agent found by pred
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if current_agent_pos == current_prey_pos: # agent found prey, agent wins
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    if (timesteps > t_hangs):
      return("agent timesteps!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])

    # update prey beliefs
    state = False
    pos_where_prey_not_there.append(current_agent_pos)
    prey_beliefs = update_belief_survey(prey_beliefs, current_agent_pos, state, graph,node_degree,pos_where_prey_not_there)
    
    # update pred beliefs
    state = False
    pos_where_pred_not_there.append(current_agent_pos)
    pred_beliefs = update_belief_survey(pred_beliefs, current_agent_pos, state, graph,node_degree,pos_where_pred_not_there)

    # prey moves
    if(len(prey_pos_list)<= 0):
      current_prey_pos = move_prey(graph, current_prey_pos)
    else:
      prey_pos_list.pop()
    pos_where_prey_not_there = []


    # update beliefs after prey moves
    if (current_prey_pos==current_agent_pos):
      return("agent wins!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      prey_beliefs = update_belief_move(prey_beliefs, graph, node_degree)

    #print("update beliefs after prey moves:",sum(prey_beliefs))
    #print(prey_beliefs)

    # predator moves
    current_pred_pos = move_distracted_predator(graph, current_pred_pos, current_agent_pos)
    # check if pred catches agent
    if (current_pred_pos==current_agent_pos): #agent and pred in same cell, agent loses
      return("agent loses!",[current_agent_pos, current_prey_pos, current_pred_pos,correctly_identified_prey_pos/timesteps, correctly_identified_pred_pos/timesteps])
      break
    else:
      # update pred beliefs
      pred_beliefs = update_belief_after_pred_move(pred_beliefs, graph, node_degree,current_agent_pos)


    pos_where_pred_not_there = [] 



# Main

In [45]:
t_hangs = 5000
prey_pos_list =[]
ENABLE_PRINT = False
simulations = 1
A0_wins,A1_wins,A2_wins,A3_wins,A4_wins,A5_wins,A6_wins,A7_wins,A8_wins,A7_fd_wins,A8_fd_wins,A7_fdN_wins,A8_fdN_wins = [0,0,0,0,0,0,0,0,0,0,0,0,0]
A0_hangs,A1_hangs,A2_hangs,A3_hangs,A4_hangs,A5_hangs,A6_hangs,A7_hangs,A8_hangs,A7_fd_hangs,A8_fd_hangs,A7_fdN_hangs,A8_fdN_hangs = [0,0,0,0,0,0,0,0,0,0,0,0,0]
A0_loss,A1_loss,A2_loss,A3_loss,A4_loss,A5_loss,A6_loss,A7_loss,A8_loss,A7_fd_loss,A8_fd_loss,A7_fdN_loss,A8_fdN_loss = [0,0,0,0,0,0,0,0,0,0,0,0,0]
p3,p4,pd5,pd6,p7,pd7,p8,pd8,p7fd,pd7fd,p8fd,pd8fd,p7fdN,pd7fdN,p8fdN,pd8fdN = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
A4N_wins,A4N_hangs = [0,0]



for i in range(simulations):
  graph, node_degree, c = create_graph()
  # print("graph:",graph)
  current_agent_pos , current_prey_pos , current_pred_pos = initilise_prey_pred_agent()
  # print([current_agent_pos, current_prey_pos, current_pred_pos])

  prey_pos_list =[]

  state, vals0 = agent0( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A0_wins = A0_wins + 1
  if state == "agent hangs!":
    A0_hangs = A0_hangs + 1
  if state == "agent loses!":
    A0_loss += 1

  state, vals1 = agent1( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A1_wins = A1_wins + 1
  if state == "agent hangs!":
    A1_hangs = A1_hangs + 1
  if state == "agent loses!":
    A1_loss += 1


  state, vals2 = agent2( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A2_wins = A2_wins + 1
  if state == "agent hangs!":
    A2_hangs = A2_hangs + 1
  if state == "agent loses!":
    A2_loss += 1

  prey_pos_list =[]

  state, vals3 = agent3( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A3_wins = A3_wins + 1
  if state == "agent hangs!":
    A3_hangs = A3_hangs + 1
  if state == "agent loses!":
    A3_loss += 1
  p3.append(vals3[-1])

  state, vals4 = agent4( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A4_wins = A4_wins + 1
  if state == "agent hangs!":
    A4_hangs = A4_hangs + 1
  if state == "agent loses!":
    A4_loss += 1
  p4.append(vals4[-1])
  

  prey_pos_list =[]

  state, vals5 = agent5( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A5_wins = A5_wins + 1
  if state == "agent hangs!":
    A5_hangs = A5_hangs + 1
  if state == "agent loses!":
    A5_loss += 1
  pd5.append(vals5[-1])

  state, vals6 = agent6( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A6_wins = A6_wins + 1
  if state == "agent hangs!":
    A6_hangs = A6_hangs + 1
  if state == "agent loses!":
    A6_loss += 1
  pd6.append(vals6[-1])

  prey_pos_list =[]

  state, vals7 = agent7( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A7_wins = A7_wins + 1
  if state == "agent hangs!":
    A7_hangs = A7_hangs + 1
  if state == "agent loses!":
    A7_loss += 1
  p7.append(vals7[-2])
  pd7.append(vals7[-1])

  state, vals8 = agent8( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A8_wins = A8_wins + 1
  if state == "agent hangs!":
    A8_hangs = A8_hangs + 1
  if state == "agent loses!":
    A8_loss += 1
  p8.append(vals8[-2])
  pd8.append(vals8[-1])

  prey_pos_list =[]

  state, vals7_fd = agent7_faulty_drone( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A7_fd_wins = A7_fd_wins + 1
  if state == "agent hangs!":
    A7_fd_hangs = A7_fd_hangs + 1
  if state == "agent loses!":
    A8_fd_loss += 1
  p7fd.append(vals7_fd[-2])
  pd7fd.append(vals7_fd[-1])

  state, vals8_fd = agent8_faulty_drone( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A8_fd_wins = A8_fd_wins + 1
  if state == "agent hangs!":
    A8_fd_hangs = A8_fd_hangs + 1
  if state == "agent loses!":
    A8_fd_loss += 1
  p8fd.append(vals8_fd[-2])
  pd8fd.append(vals8_fd[-1])

  state, vals7_fdN = agent7_faulty_drone_with_new_belief_update( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A7_fdN_wins = A7_fdN_wins + 1
  if state == "agent hangs!":
    A7_fdN_hangs = A7_fdN_hangs + 1
  if state == "agent loses!":
    A7_fdN_loss += 1
  p7fdN.append(vals7_fdN[-2])
  pd7fdN.append(vals7_fdN[-1])

  state, vals8_fdN = agent8_faulty_drone_with_new_belief_update( current_agent_pos , current_prey_pos , current_pred_pos)
  if state == "agent wins!":
    A8_fdN_wins = A8_fdN_wins + 1
  if state == "agent hangs!":
    A8_fdN_hangs = A8_fdN_hangs + 1
  if state == "agent loses!":
    A8_fdN_loss += 1
  p8fdN.append(vals8_fdN[-2])
  pd8fdN.append(vals8_fdN[-1])


print("AGENT 1\nSuccess:",(A1_wins/(simulations)) * 100," Hangs:",A1_hangs) 
print("AGENT 2\nSuccess:",(A2_wins/(simulations)) * 100," Hangs:",A2_hangs) 
print("AGENT 3\nSuccess:",(A3_wins/(simulations)) * 100," Hangs:",A3_hangs," Correct prediction prey",sum(p3)) 
print("AGENT 4\nSuccess:",(A4_wins/(simulations)) * 100," Hangs:",A4_hangs," Correct prediction prey:",(sum(p4)/simulations)*100) 
print("AGENT 5\nSuccess:",(A5_wins/(simulations)) * 100," Hangs:",A5_hangs," Correct prediction pred:",(sum(pd5)/simulations)*100)
print("AGENT 6\nSuccess:",(A6_wins/(simulations)) * 100," Hangs:",A6_hangs," Correct prediction pred:",(sum(pd6)/simulations)*100)
print("AGENT 7\nSuccess:",(A7_wins/(simulations)) * 100," Hangs:",A7_hangs," Correct prediction prey:",(sum(p7)/simulations)*100," Correct prediction pred:",(sum(pd7)/simulations)*100)
print("AGENT 8\nSuccess:",(A8_wins/(simulations)) * 100," Hangs:",A8_hangs," Correct prediction prey:",(sum(p8)/simulations)*100," Correct prediction pred:",(sum(pd8)/simulations)*100)
print("AGENT 7 faulty drone\nSuccess:",A7_fd_wins," Hangs:",A7_fd_hangs," Correct prediction prey:",(sum(p7fd)/simulations)*100," Correct prediction pred:",(sum(pd7fd)/simulations)*100)
print("AGENT 8 faulty drone\nSuccess:",A8_fd_wins," Hangs:",A8_fd_hangs," Correct prediction prey:",(sum(p8fd)/simulations)*100," Correct prediction pred:",(sum(pd8fd)/simulations)*100)
print("AGENT 7 faulty drone with new belief updates\nSuccess:",A7_fdN_wins," Hangs:",A7_fdN_hangs," Correct prediction prey:",(sum(p7fdN)/simulations)*100," Correct prediction pred:",(sum(pd7fdN)/simulations)*100)
print("AGENT 8 faulty drone with new belief updates\nSuccess:",A8_fdN_wins," Hangs:",A8_fdN_hangs," Correct prediction prey:",(sum(p8fdN)/simulations)*100," Correct prediction pred:",(sum(pd8fdN)/simulations)*100)



AGENT 1
Success: 83.33333333333334  Hangs: 0
AGENT 2
Success: 98.06666666666666  Hangs: 0
AGENT 3
Success: 81.69999999999999  Hangs: 0  Correct prediction prey: 11.4100707340379
AGENT 4
Success: 96.33333333333334  Hangs: 0  Correct prediction prey: 11.5855673403751
AGENT 5
Success: 81.69999999999999  Hangs: 0  Correct prediction pred: 55.945891530002925
AGENT 6
Success: 89.8  Hangs: 0  Correct prediction pred: 55.996976372645115
AGENT 7
Success: 75.4  Hangs: 0  Correct prediction prey: 1.6680821045238738  Correct prediction pred: 56.36290186083507
AGENT 8
Success: 83.39999999999999  Hangs: 0  Correct prediction prey: 1.65877026662225  Correct prediction pred: 56.08289865820018
AGENT 7 faulty drone
Success: 55.18559499522547  Hangs: 0  Correct prediction prey: 1.6081300591638437  Correct prediction pred: 55.16259034758334
AGENT 8 faulty drone
Success: 60.62519516161847  Hangs: 0  Correct prediction prey: 1.5944433060412254  Correct prediction pred: 55.248742257197403
AGENT 7 faulty dron